We currently have a model that can deal with one character of previous context. But the precition was not good enough.

* More context? way too many rows => expensive to calculate
* MLP

In [15]:
import torch
import torch.nn.functional as F

# Data

In [2]:
names = open("names.txt").read().splitlines()
names[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [3]:
vocab = sorted(set(''.join(names)))
vocab = ["."] + vocab
len(vocab)

27

In [4]:
stoi = {s:i for i, s in enumerate(vocab)}
itos = {i:s for s, i in stoi.items()}

In [5]:
block_size = 5 # context length
xs, ys = [], []

for name in names[:5]:
    context = [0] * block_size
    print(name)
    for ch in name+".":
        idx = stoi[ch]
        xs.append(context)
        ys.append(idx)
        print(f"{''.join([itos[i] for i in context])} -------> {itos[idx]}")
        context = context[1:] + [idx]
xs, ys = torch.tensor(xs), torch.tensor(ys)

emma
..... -------> e
....e -------> m
...em -------> m
..emm -------> a
.emma -------> .
olivia
..... -------> o
....o -------> l
...ol -------> i
..oli -------> v
.oliv -------> i
olivi -------> a
livia -------> .
ava
..... -------> a
....a -------> v
...av -------> a
..ava -------> .
isabella
..... -------> i
....i -------> s
...is -------> a
..isa -------> b
.isab -------> e
isabe -------> l
sabel -------> l
abell -------> a
bella -------> .
sophia
..... -------> s
....s -------> o
...so -------> p
..sop -------> h
.soph -------> i
sophi -------> a
ophia -------> .


In [6]:
xs.shape, xs.dtype, ys.shape, ys.dtype

(torch.Size([32, 5]), torch.int64, torch.Size([32]), torch.int64)

# Embedding

Our vocab size is 27 and we want to map them into 2-D space. In original paper from Bengio, they have 17,000 words and then map them into 30-D space.

In [16]:
C = torch.randn([27, 2])
C

tensor([[-0.8262, -0.0732],
        [-1.0438,  0.3313],
        [-0.7443, -0.0562],
        [ 0.0032, -0.9635],
        [ 1.3140, -0.1375],
        [-0.4448,  0.6388],
        [ 0.6052,  1.8997],
        [-0.7271, -0.4932],
        [-0.7053,  0.5348],
        [ 0.6374, -0.1047],
        [-1.6623, -1.9371],
        [-1.0474,  0.2313],
        [-1.4575, -0.2714],
        [-1.1575, -0.4749],
        [ 0.2037, -2.8663],
        [-0.6498, -0.6538],
        [-0.9157,  0.6796],
        [ 0.3786, -0.5950],
        [-0.3459,  1.3806],
        [-0.3691, -0.2993],
        [ 0.6855, -0.3610],
        [ 0.9752, -0.0268],
        [ 0.1188,  1.1797],
        [ 1.5071, -0.9910],
        [ 0.9470, -0.1562],
        [-1.6616, -0.0554],
        [ 0.3088,  0.7594]])

In [20]:
xenc = F.one_hot(torch.tensor(5), num_classes=27).float()
# which is equal to C[5]
# 5 th row will be plucked out.
C[5], C[5] == xenc @ C

(tensor([-0.4448,  0.6388]), tensor([True, True]))

In [30]:
xs

tensor([[ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  5],
        [ 0,  0,  0,  5, 13],
        [ 0,  0,  5, 13, 13],
        [ 0,  5, 13, 13,  1],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 15],
        [ 0,  0,  0, 15, 12],
        [ 0,  0, 15, 12,  9],
        [ 0, 15, 12,  9, 22],
        [15, 12,  9, 22,  9],
        [12,  9, 22,  9,  1],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  1],
        [ 0,  0,  0,  1, 22],
        [ 0,  0,  1, 22,  1],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  9],
        [ 0,  0,  0,  9, 19],
        [ 0,  0,  9, 19,  1],
        [ 0,  9, 19,  1,  2],
        [ 9, 19,  1,  2,  5],
        [19,  1,  2,  5, 12],
        [ 1,  2,  5, 12, 12],
        [ 2,  5, 12, 12,  1],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 19],
        [ 0,  0,  0, 19, 15],
        [ 0,  0, 19, 15, 16],
        [ 0, 19, 15, 16,  8],
        [19, 15, 16,  8,  9],
        [15, 16,  8,  9,  1]])

In [34]:
C[xs].shape, C[xs][1, 4], C[xs][1, 4] == C[5] 

(torch.Size([32, 5, 2]), tensor([-0.4448,  0.6388]), tensor([True, True]))